TSG100 - The Azure Arc Postgres troubleshooter
==============================================

Description
-----------

Follow these steps to troubleshoot an Azure Arc Postgres Server.

Steps
-----

### Parameters

In [ ]:
import os

# The server to troubleshoot. If not provided
# the user will be prompted to select a server.
namespace = os.environ.get('POSTGRES_SERVER_NAMESPACE')
name = os.environ.get('POSTGRES_SERVER_NAME')

tail_lines = 50

### Common functions

Define helper functions used in this notebook.

In [ ]:
# Define `run` function for transient fault handling, suggestions on error, and scrolling updates on Windows
import sys
import os
import re
import json
import platform
import shlex
import shutil
import datetime

from subprocess import Popen, PIPE
from IPython.display import Markdown

retry_hints = {} # Output in stderr known to be transient, therefore automatically retry
error_hints = {} # Output in stderr where a known SOP/TSG exists which will be HINTed for further help
install_hint = {} # The SOP to help install the executable if it cannot be found

first_run = True
rules = None
debug_logging = False

def run(cmd, return_output=False, no_output=False, retry_count=0):
    """Run shell command, stream stdout, print stderr and optionally return output

    NOTES:

    1.  Commands that need this kind of ' quoting on Windows e.g.:

            kubectl get nodes -o jsonpath={.items[?(@.metadata.annotations.pv-candidate=='data-pool')].metadata.name}

        Need to actually pass in as '"':

            kubectl get nodes -o jsonpath={.items[?(@.metadata.annotations.pv-candidate=='"'data-pool'"')].metadata.name}

        The ' quote approach, although correct when pasting into Windows cmd, will hang at the line:
        
            `iter(p.stdout.readline, b'')`

        The shlex.split call does the right thing for each platform, just use the '"' pattern for a '
    """
    MAX_RETRIES = 5
    output = ""
    retry = False

    global first_run
    global rules

    if first_run:
        first_run = False
        rules = load_rules()

    # When running `azdata sql query` on Windows, replace any \n in """ strings, with " ", otherwise we see:
    #
    #    ('HY090', '[HY090] [Microsoft][ODBC Driver Manager] Invalid string or buffer length (0) (SQLExecDirectW)')
    #
    if platform.system() == "Windows" and cmd.startswith("azdata sql query"):
        cmd = cmd.replace("\n", " ")

    # shlex.split is required on bash and for Windows paths with spaces
    #
    cmd_actual = shlex.split(cmd)

    # Store this (i.e. kubectl, python etc.) to support binary context aware error_hints and retries
    #
    user_provided_exe_name = cmd_actual[0].lower()

    # When running python, use the python in the ADS sandbox ({sys.executable})
    #
    if cmd.startswith("python "):
        cmd_actual[0] = cmd_actual[0].replace("python", sys.executable)

        # On Mac, when ADS is not launched from terminal, LC_ALL may not be set, which causes pip installs to fail
        # with:
        #
        #    UnicodeDecodeError: 'ascii' codec can't decode byte 0xc5 in position 4969: ordinal not in range(128)
        #
        # Setting it to a default value of "en_US.UTF-8" enables pip install to complete
        #
        if platform.system() == "Darwin" and "LC_ALL" not in os.environ:
            os.environ["LC_ALL"] = "en_US.UTF-8"

    # When running `kubectl`, if AZDATA_OPENSHIFT is set, use `oc`
    #
    if cmd.startswith("kubectl ") and "AZDATA_OPENSHIFT" in os.environ:
        cmd_actual[0] = cmd_actual[0].replace("kubectl", "oc")

    # To aid supportabilty, determine which binary file will actually be executed on the machine
    #
    which_binary = None

    # Special case for CURL on Windows.  The version of CURL in Windows System32 does not work to
    # get JWT tokens, it returns "(56) Failure when receiving data from the peer".  If another instance
    # of CURL exists on the machine use that one.  (Unfortunately the curl.exe in System32 is almost
    # always the first curl.exe in the path, and it can't be uninstalled from System32, so here we
    # look for the 2nd installation of CURL in the path)
    if platform.system() == "Windows" and cmd.startswith("curl "):
        path = os.getenv('PATH')
        for p in path.split(os.path.pathsep):
            p = os.path.join(p, "curl.exe")
            if os.path.exists(p) and os.access(p, os.X_OK):
                if p.lower().find("system32") == -1:
                    cmd_actual[0] = p
                    which_binary = p
                    break

    # Find the path based location (shutil.which) of the executable that will be run (and display it to aid supportability), this
    # seems to be required for .msi installs of azdata.cmd/az.cmd.  (otherwise Popen returns FileNotFound) 
    #
    # NOTE: Bash needs cmd to be the list of the space separated values hence shlex.split.
    #
    if which_binary == None:
        which_binary = shutil.which(cmd_actual[0])

    if which_binary == None:
        if user_provided_exe_name in install_hint and install_hint[user_provided_exe_name] is not None:
            display(Markdown(f'HINT: Use [{install_hint[user_provided_exe_name][0]}]({install_hint[user_provided_exe_name][1]}) to resolve this issue.'))

        raise FileNotFoundError(f"Executable '{cmd_actual[0]}' not found in path (where/which)")
    else:   
        cmd_actual[0] = which_binary

    start_time = datetime.datetime.now().replace(microsecond=0)

    print(f"START: {cmd} @ {start_time} ({datetime.datetime.utcnow().replace(microsecond=0)} UTC)")
    print(f"       using: {which_binary} ({platform.system()} {platform.release()} on {platform.machine()})")
    print(f"       cwd: {os.getcwd()}")

    # Command-line tools such as CURL and AZDATA HDFS commands output
    # scrolling progress bars, which causes Jupyter to hang forever, to
    # workaround this, use no_output=True
    #

    # Work around a infinite hang when a notebook generates a non-zero return code, break out, and do not wait
    #
    wait = True 

    try:
        if no_output:
            p = Popen(cmd_actual)
        else:
            p = Popen(cmd_actual, stdout=PIPE, stderr=PIPE, bufsize=1)
            with p.stdout:
                for line in iter(p.stdout.readline, b''):
                    line = line.decode()
                    if return_output:
                        output = output + line
                    else:
                        if cmd.startswith("azdata notebook run"): # Hyperlink the .ipynb file
                            regex = re.compile('  "(.*)"\: "(.*)"') 
                            match = regex.match(line)
                            if match:
                                if match.group(1).find("HTML") != -1:
                                    display(Markdown(f' - "{match.group(1)}": "{match.group(2)}"'))
                                else:
                                    display(Markdown(f' - "{match.group(1)}": "[{match.group(2)}]({match.group(2)})"'))

                                    wait = False
                                    break # otherwise infinite hang, have not worked out why yet.
                        else:
                            print(line, end='')
                            if rules is not None:
                                apply_expert_rules(line)

        if wait:
            p.wait()
    except FileNotFoundError as e:
        if install_hint is not None:
            display(Markdown(f'HINT: Use {install_hint} to resolve this issue.'))

        raise FileNotFoundError(f"Executable '{cmd_actual[0]}' not found in path (where/which)") from e

    exit_code_workaround = 0 # WORKAROUND: azdata hangs on exception from notebook on p.wait()

    if not no_output:
        for line in iter(p.stderr.readline, b''):
            try:
                line_decoded = line.decode()
            except UnicodeDecodeError:
                # NOTE: Sometimes we get characters back that cannot be decoded(), e.g.
                #
                #   \xa0
                #
                # For example see this in the response from `az group create`:
                #
                # ERROR: Get Token request returned http error: 400 and server 
                # response: {"error":"invalid_grant",# "error_description":"AADSTS700082: 
                # The refresh token has expired due to inactivity.\xa0The token was 
                # issued on 2018-10-25T23:35:11.9832872Z
                #
                # which generates the exception:
                #
                # UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa0 in position 179: invalid start byte
                #
                print("WARNING: Unable to decode stderr line, printing raw bytes:")
                print(line)
                line_decoded = ""
                pass
            else:

                # azdata emits a single empty line to stderr when doing an hdfs cp, don't
                # print this empty "ERR:" as it confuses.
                #
                if line_decoded == "":
                    continue
                
                print(f"STDERR: {line_decoded}", end='')

                if line_decoded.startswith("An exception has occurred") or line_decoded.startswith("ERROR: An error occurred while executing the following cell"):
                    exit_code_workaround = 1

                # inject HINTs to next TSG/SOP based on output in stderr
                #
                if user_provided_exe_name in error_hints:
                    for error_hint in error_hints[user_provided_exe_name]:
                        if line_decoded.find(error_hint[0]) != -1:
                            display(Markdown(f'HINT: Use [{error_hint[1]}]({error_hint[2]}) to resolve this issue.'))

                # apply expert rules (to run follow-on notebooks), based on output
                #
                if rules is not None:
                    apply_expert_rules(line_decoded)

                # Verify if a transient error, if so automatically retry (recursive)
                #
                if user_provided_exe_name in retry_hints:
                    for retry_hint in retry_hints[user_provided_exe_name]:
                        if line_decoded.find(retry_hint) != -1:
                            if retry_count < MAX_RETRIES:
                                print(f"RETRY: {retry_count} (due to: {retry_hint})")
                                retry_count = retry_count + 1
                                output = run(cmd, return_output=return_output, retry_count=retry_count)

                                if return_output:
                                    return output
                                else:
                                    return

    elapsed = datetime.datetime.now().replace(microsecond=0) - start_time

    # WORKAROUND: We avoid infinite hang above in the `azdata notebook run` failure case, by inferring success (from stdout output), so
    # don't wait here, if success known above
    #
    if wait: 
        if p.returncode != 0:
            raise SystemExit(f'Shell command:\n\n\t{cmd} ({elapsed}s elapsed)\n\nreturned non-zero exit code: {str(p.returncode)}.\n')
    else:
        if exit_code_workaround !=0 :
            raise SystemExit(f'Shell command:\n\n\t{cmd} ({elapsed}s elapsed)\n\nreturned non-zero exit code: {str(exit_code_workaround)}.\n')

    print(f'\nSUCCESS: {elapsed}s elapsed.\n')

    if return_output:
        return output

def load_json(filename):
    """Load a json file from disk and return the contents"""

    with open(filename, encoding="utf8") as json_file:
        return json.load(json_file)

def load_rules():
    """Load any 'expert rules' from the metadata of this notebook (.ipynb) that should be applied to the stderr of the running executable"""

    # Load this notebook as json to get access to the expert rules in the notebook metadata.
    #
    try:
        j = load_json("tsg100-troubleshoot-postgres.ipynb")
    except:
        pass # If the user has renamed the book, we can't load ourself.  NOTE: Is there a way in Jupyter, to know your own filename?
    else:
        if "metadata" in j and \
            "azdata" in j["metadata"] and \
            "expert" in j["metadata"]["azdata"] and \
            "expanded_rules" in j["metadata"]["azdata"]["expert"]:

            rules = j["metadata"]["azdata"]["expert"]["expanded_rules"]

            rules.sort() # Sort rules, so they run in priority order (the [0] element).  Lowest value first.

            # print (f"EXPERT: There are {len(rules)} rules to evaluate.")

            return rules

def apply_expert_rules(line):
    """Determine if the stderr line passed in, matches the regular expressions for any of the 'expert rules', if so
    inject a 'HINT' to the follow-on SOP/TSG to run"""

    global rules

    for rule in rules:
        notebook = rule[1]
        cell_type = rule[2]
        output_type = rule[3] # i.e. stream or error
        output_type_name = rule[4] # i.e. ename or name 
        output_type_value = rule[5] # i.e. SystemExit or stdout
        details_name = rule[6]  # i.e. evalue or text 
        expression = rule[7].replace("\\*", "*") # Something escaped *, and put a \ in front of it!

        if debug_logging:
            print(f"EXPERT: If rule '{expression}' satisfied', run '{notebook}'.")

        if re.match(expression, line, re.DOTALL):

            if debug_logging:
                print("EXPERT: MATCH: name = value: '{0}' = '{1}' matched expression '{2}', therefore HINT '{4}'".format(output_type_name, output_type_value, expression, notebook))

            match_found = True

            display(Markdown(f'HINT: Use [{notebook}]({notebook}) to resolve this issue.'))



print('Common functions defined successfully.')

# Hints for binary (transient fault) retry, (known) error and install guide
#
retry_hints = {'kubectl': ['A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond']}
error_hints = {'kubectl': [['no such host', 'TSG010 - Get configuration contexts', '../monitor-k8s/tsg010-get-kubernetes-contexts.ipynb'], ['No connection could be made because the target machine actively refused it', 'TSG056 - Kubectl fails with No connection could be made because the target machine actively refused it', '../repair/tsg056-kubectl-no-connection-could-be-made.ipynb']]}
install_hint = {'kubectl': ['SOP036 - Install kubectl command line interface', '../install/sop036-install-kubectl.ipynb']}

### Get Postgres server

In [ ]:
# Sets the 'server' variable to the spec of the Postgres server

import math

# If a server was provided, get it
if namespace and name:
    server = json.loads(run(f'kubectl get dbs -n {namespace} {name} -o json', return_output=True))
else:
    # Otherwise prompt the user to select a server
    servers = json.loads(run(f'kubectl get dbs --all-namespaces -o json', return_output=True))['items']
    if not servers:
        raise Exception('No Postgres servers found')

    full_name = lambda s: f'{s["metadata"]["namespace"]}.{s["metadata"]["name"]}'
    servers.sort(key=lambda s: full_name(s))

    pad = math.floor(math.log10(len(servers)) + 1) + 3
    for i, s in enumerate(servers):
        print(f'{f"[{i+1}]":<{pad}}{full_name(s)}')

    while True:
        try:
            i = int(input('Enter the index of a server to troubleshoot: '))
        except ValueError:
            continue

        if i >= 1 and i <= len(servers):
            server = servers[i-1]
            namespace = server['metadata']['namespace']
            name = server['metadata']['name']
            break

display(Markdown(f'#### Got server {namespace}.{name}'))

### Summarize all resources

In [ ]:
uid = server['metadata']['uid']

display(Markdown(f'#### Server summary'))
run(f'kubectl get dbs -n {namespace} {name}')

display(Markdown(f'#### Resource summary'))
run(f'kubectl get pods,pvc,svc,ep -n {namespace} -l dusky.microsoft.com/serviceId={uid}')

### Troubleshoot the server

In [ ]:
display(Markdown(f'#### Troubleshooting server {namespace}.{name}'))
run(f'kubectl describe dbs -n {namespace} {name}')

### Troubleshoot the pods

In [ ]:
pods = json.loads(run(f'kubectl get pods -n {namespace} -l dusky.microsoft.com/serviceId={uid} -o json', return_output=True))['items']

# Summarize and describe each pod
for pod in pods:
    pod_name = pod['metadata']['name']
    pod_phase = pod['status']['phase']
    con_statuses = pod['status'].get('containerStatuses', [])
    num_con_ready = sum(1 for c in con_statuses if c['ready'])
    num_cons = len(pod['spec']['containers'])
    num_restarts = sum(c['restartCount'] for c in con_statuses)

    display(Markdown(f'#### Troubleshooting pod {namespace}.{pod_name}\n'
                     f'#### {pod_phase} with {num_con_ready}/{num_cons} '
                     f'containers ready and {num_restarts} restarts'))

    run(f'kubectl describe pod -n {namespace} {pod_name}')

### Troubleshoot the containers

In [ ]:
# Summarize and get logs from each container
for pod in pods:
    pod_name = pod['metadata']['name']
    cons = pod['spec']['containers']
    con_statuses = pod['status'].get('containerStatuses', [])
    display(Markdown(f'#### Troubleshooting {len(cons)} container{"" if len(cons) < 2 else "s"} '
                     f'containers for pod {namespace}.{pod_name}'))

    for i, con in enumerate(cons):
        con_name = con['name']
        con_status = next((c for c in con_statuses if c['name'] == con_name), {})
        con_started = con_status.get('started', False)
        con_ready = con_status.get('ready', False)
        con_restarts = con_status.get('restartCount', 0)

        display(Markdown(f'#### Troubleshooting container {namespace}.{pod_name}/{con_name} ({i+1}/{len(cons)})\n'
                         f'#### {"S" if con_started else "Not s"}tarted and '
                         f'{"" if con_ready else "not "}ready with {con_restarts} restarts'))

        run(f'kubectl logs -n {namespace} {pod_name} {con_name} --tail {tail_lines}')

        # Get logs from the previous terminated container if one exists
        if con_restarts > 0:
            display(Markdown(f'#### Logs from previous terminated container {namespace}.{pod_name}/{con_name}'))
            run(f'kubectl logs -n {namespace} {pod_name} {con_name} --tail {tail_lines} --previous')

### Troubleshoot the PersistentVolumeClaims

In [ ]:
display(Markdown(f'#### Troubleshooting PersistentVolumeClaims'))
run(f'kubectl describe pvc -n {namespace} -l dusky.microsoft.com/serviceId={uid}')

In [ ]:
print('Notebook execution complete.')